In [1]:
import keras
import numpy as np
import pandas as pd
import matchzoo as mz

Using TensorFlow backend.
Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 2118/2118 [00:00<00:00, 2876.80it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4750.00it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 1614976/1614976 [00:00<00:00, 4180880.97it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 2118/2118 [00:00<00:00, 2701.74it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 18841/18841 [00:26<00:00, 698.00it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 122/122 [00:00<00:00, 2522.88it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 1115/1115 [00:01<00:00, 716.62it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 237/237 [00:00<00:00, 2970.25it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 2300/2300 [00:03<00:00, 717.43it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.DSSM()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DSSM.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 9644)         0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 9644)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 300)          2893500     text_left[0][0]                  
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 300)          2893500     text_right[0][0]                 
_______________________________________________________________________________

In [7]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [8]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=1, batch_size=64, shuffle=True)
len(train_generator)

32

In [9]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=5, use_multiprocessing=False)

Epoch 1/20
32/32 [==============================] - 6s 180ms/step - loss: 0.9847
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.433040 - normalized_discounted_cumulative_gain@5(0):0.507081 - mean_average_precision(0):0.465983
Epoch 2/20
32/32 [==============================] - 3s 108ms/step - loss: 0.9486
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.557635 - normalized_discounted_cumulative_gain@5(0):0.603704 - mean_average_precision(0):0.581350
Epoch 3/20
32/32 [==============================] - 3s 109ms/step - loss: 0.9373
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.464820 - normalized_discounted_cumulative_gain@5(0):0.534015 - mean_average_precision(0):0.499414
Epoch 4/20
32/32 [==============================] - 4s 109ms/step - loss: 0.9108
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.536365 - normalized_discounted_cumulative_gain@5(0):0.582650 - mean_average_precision(0):0.560792
Epoch 5/20
3